In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
airline_df = pd.read_csv("cleaned_airline_2018.csv")

abe_weather_df = pd.read_csv("abe_cleaned_weather_2018.csv")
atl_weather_df = pd.read_csv("atl_cleaned_weather_2018.csv")

In [3]:
airline_df.head()

,FL_DATE,ORIGIN,DEST,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,Alaska Airlines,Allegiant Air,American Airlines,Delta Airlines,...,Hawaiian Airlines,JetBlue Airways,Mesa Airline,PSA Airlines,Republic Airways,SkyWest Airlines,Southwest Airlines,Spirit Airlines,United Airlines,Virgin America
0,2018-01-01,EWR,DEN,268.0,250.0,1605.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,2018-01-01,LAS,SFO,99.0,83.0,414.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2018-01-01,SNA,DEN,134.0,126.0,846.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2018-01-01,RSW,ORD,190.0,182.0,1120.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,2018-01-01,ORD,ALB,112.0,106.0,723.0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [4]:
abe_weather_df.head()

,time,precipitation_sum (mm),rain_sum (mm),snowfall_sum (cm),windspeed_10m_max (km/h),windgusts_10m_max (km/h),et0_fao_evapotranspiration (mm)
0,2018-01-01,0.0,0.0,0.00,17.9,43.2,0.81
1,2018-01-02,0.0,0.0,0.00,16.0,38.9,0.89
2,2018-01-03,0.0,0.0,0.00,9.3,22.7,0.85
3,2018-01-04,10.2,0.0,7.28,29.8,67.0,0.74
4,2018-01-05,0.0,0.0,0.00,30.9,67.3,0.92


In [5]:
airline_df[airline_df['ORIGIN'] == 'ABE']['DEST'].value_counts()

ATL    960
DTW    912
CLT    911
SFB    441
ORD    345
PIE    157
PGD    144
MYR     84
FLL     72
PHL     40
Name: DEST, dtype: int64

In [6]:
abe_atl_airline_df = airline_df[(airline_df['ORIGIN'] == 'ABE') & (airline_df['DEST'] == 'ATL')]
abe_atl_airline_df.head()

,FL_DATE,ORIGIN,DEST,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,Alaska Airlines,Allegiant Air,American Airlines,Delta Airlines,...,Hawaiian Airlines,JetBlue Airways,Mesa Airline,PSA Airlines,Republic Airways,SkyWest Airlines,Southwest Airlines,Spirit Airlines,United Airlines,Virgin America
3936,2018-01-01,ABE,ATL,149.0,170.0,692.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19725,2018-01-02,ABE,ATL,141.0,119.0,692.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19733,2018-01-02,ABE,ATL,149.0,134.0,692.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40933,2018-01-03,ABE,ATL,139.0,144.0,692.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54149,2018-01-03,ABE,ATL,140.0,159.0,692.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [53]:
weather_cols = list(abe_weather_df.columns)
abe_atl_airline_df[weather_cols] = 0

# exclude time
for col in weather_cols[1:]:
    def exsert_weather(row):
        return abe_weather_df[abe_weather_df['time'] == row['FL_DATE']][col]
    df = abe_atl_airline_df.apply(exsert_weather, axis=1)
    break

abe_atl_airline_df.head()

,FL_DATE,ORIGIN,DEST,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,Alaska Airlines,Allegiant Air,American Airlines,Delta Airlines,...,Spirit Airlines,United Airlines,Virgin America,precipitation_sum (mm),rain_sum (mm),snowfall_sum (cm),windspeed_10m_max (km/h),windgusts_10m_max (km/h),et0_fao_evapotranspiration (mm),time
3936,2018-01-01,ABE,ATL,149.0,170.0,692.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19725,2018-01-02,ABE,ATL,141.0,119.0,692.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19733,2018-01-02,ABE,ATL,149.0,134.0,692.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40933,2018-01-03,ABE,ATL,139.0,144.0,692.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54149,2018-01-03,ABE,ATL,140.0,159.0,692.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [39]:
abe_atl_airline_df.head()

,FL_DATE,ORIGIN,DEST,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,Alaska Airlines,Allegiant Air,American Airlines,Delta Airlines,...,Spirit Airlines,United Airlines,Virgin America,precipitation_sum (mm),rain_sum (mm),snowfall_sum (cm),windspeed_10m_max (km/h),windgusts_10m_max (km/h),et0_fao_evapotranspiration (mm),time
3936,2018-01-01,ABE,ATL,149.0,170.0,692.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19725,2018-01-02,ABE,ATL,141.0,119.0,692.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19733,2018-01-02,ABE,ATL,149.0,134.0,692.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40933,2018-01-03,ABE,ATL,139.0,144.0,692.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54149,2018-01-03,ABE,ATL,140.0,159.0,692.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [51]:
df

,0,1,2,3,4,5,6,7,8,9,...,355,356,357,358,359,360,361,362,363,364
3936,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19725,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19733,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40933,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54149,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7031296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
7031312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
7057698,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
7057715,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
